In [1]:
# Import libraries
import os
import datetime
import requests
import io
import shutil
from glob import glob

import earthpy as et
import numpy as np
import pandas as pd

import kba_thresh_sa_scripts as ks

In [2]:
# Define a filepath to 'earth-analytics/data/kba_thresh_sa' directory
# (kba_thresh_sa is an abbreviation of KBA Threshold Sensitivity Analysis)
data_path = os.path.normpath(os.path.join(et.io.HOME, 
                                          'earth-analytics', 
                                          'data', 
                                          'kba_thresh_sa'))

# Check if 'kba_thresh_sa' directory exists (create if needed), 
# then change working directory to 'earth-analytics/data/kba_thresh_sa'
if os.path.exists(data_path):
 print('Working directory is set to earth-analytics/data/kba_th_sa.')
else:
 print("Creating new directory; working directory set to newly created "
       "'earth-analytics/data/kba_thresh_sa'.")
 os.makedirs(data_path)
 
os.chdir(data_path)


Working directory is set to earth-analytics/data/kba_th_sa.


In [3]:
# Download the csv file stored on GitHub repository 
# (contains info on selected ecosystems taken from LF_EVT_2020_README file, 
# with an added 'Short_Name' field that is used as index)
url_ecoinfo = "https://raw.githubusercontent.com/csandberg303/kba-threshold-sensitivity-analysis/main/assets/data/from_LF_EVT_2020_README.csv"
ecoinfo = requests.get(url_ecoinfo).content

# Reading the downloaded content and turning it into a pandas dataframe
ecoinfo_df = pd.read_csv(
    io.StringIO(ecoinfo.decode('utf-8'))).set_index("Short_Name")

ecoinfo_df

,OID,Value,Count_30m,US_hectare,US_km2,EVT_Name_1,LFRDB,elcode,element_gl,NatureServ,...,A3_FINAL,B1_FINAL,B2_FINAL,C3_FINAL,D3_FINAL,RLE_FINAL,GRANK_EQUI,RED,GREEN,BLUE
Short_Name,,,,,,,,,,,,,,,,,,,,,
prairie,132,7142,106116,9550,96,Columbia Basin Palouse Prairie,7142,CES304.792,722880,Columbia Basin Palouse Prairie,...,CR,LC,LC,EN,VU,CR,G1,204,252,105
foothill,137,7147,4546277,409165,4092,Western Great Plains Foothill and Piedmont Gra...,7147,CES303.817,722856,Western Great Plains Foothill and Piedmont Gra...,...,VU,LC,LC,NT,LC,VU,G3,218,238,243
mesic,224,7322,1090956,98186,982,Crowley's Ridge Mesic Loess Slope Forest,7322,CES203.079,798100,Crowley's Ridge Mesic Loess Slope Forest,...,LC,EN,EN,VU,CR,CR (EN-CR),G1G2,144,201,143
bluff,229,7327,2050154,184514,1845,East Gulf Coastal Plain Northern Loess Bluff F...,7327,CES203.481,723105,East Gulf Coastal Plain Northern Loess Bluff F...,...,EN,LC,LC,VU-EN,EN,EN,G2,149,143,26
pine,244,7346,5015841,451426,4514,Atlantic Coastal Plain Fall-line Sandhills Lon...,7346,CES203.254,723231,Atlantic Coastal Plain Fall-line Sandhills Lon...,...,EN,LC,LC,VU,CR,EN (EN-CR),G1G2,70,96,32
tallgrass,314,7421,10225903,920331,9203,Central Tallgrass Prairie,7421,CES205.683,722976,Central Tallgrass Prairie,...,CR,LC,LC,DD,CR,CR,G1,243,201,28
dune,323,7431,19717,1775,18,Southwest Florida Dune and Coastal Grassland,7431,CES203.539,723063,Southwest Florida Dune and Coastal Grassland,...,DD,LC,LC,NE,CR,CR,G1,245,252,179
dome,335,7447,900234,81021,810,South Florida Cypress Dome,7447,CES411.365,723151,South Florida Cypress Dome,...,DD,VU,LC,DD,LC,VU,G3,54,163,120
marsh,676,9197,1634510,147106,1471,Northern Atlantic Coastal Plain Tidal Salt Marsh,9197,CES203.519,723073,Northern Atlantic Coastal Plain Tidal Salt Marsh,...,EN,LC,LC,DD,EN,EN,G2,131,173,223


In [4]:
# Add 2 columns to 'ecoinfo_df'

# 1st column - Add column 'Type' (needed for CLUZ addin input file 
#'targets.csv'; might not be needed for marxanconpy) Uses np.select to assign 
# a number (1 or 2), based upon the string seen in the 'RLE_FINAL' column
# (Type = 1 if 'CR', 'CR (CR-EN)', 'EN (CR-EN) or 'EN'; Type = 2 if 'VU')

# create a list of conditions
type_conditions = [(ecoinfo_df['RLE_FINAL'] == 'CR'), 
                  (ecoinfo_df['RLE_FINAL'] == 'CR (EN-CR)'),
                  (ecoinfo_df['RLE_FINAL'] == 'EN'),
                  (ecoinfo_df['RLE_FINAL'] == 'EN (EN-CR)'),
                  (ecoinfo_df['RLE_FINAL'] == 'VU')]

# create a list of the values to assign for each condition
type_values = [1, 1, 1, 1, 2]

# create new column using np.select to assign values using lists as arguments
ecoinfo_df['Type'] = np.select(type_conditions, type_values)

# 2nd column - Add column 'Current_IUCN_TH'. Uses np.select to assign a 
# threshold percentage, based upon the column 'Type' (5% if 1, 10% if 2)

# create a list of conditions
current_threshold_conditions = [(ecoinfo_df['Type'] == 1), 
                               (ecoinfo_df['Type'] == 2)]

# create a list of the values to assign for each condition
current_threshold_values = [.05, .10]

# create new column using np.select to assign values using lists as arguments
ecoinfo_df['Current_IUCN_TH'] = np.select(
    current_threshold_conditions, current_threshold_values)

ecoinfo_df

,OID,Value,Count_30m,US_hectare,US_km2,EVT_Name_1,LFRDB,elcode,element_gl,NatureServ,...,B2_FINAL,C3_FINAL,D3_FINAL,RLE_FINAL,GRANK_EQUI,RED,GREEN,BLUE,Type,Current_IUCN_TH
Short_Name,,,,,,,,,,,,,,,,,,,,,
prairie,132,7142,106116,9550,96,Columbia Basin Palouse Prairie,7142,CES304.792,722880,Columbia Basin Palouse Prairie,...,LC,EN,VU,CR,G1,204,252,105,1,0.05
foothill,137,7147,4546277,409165,4092,Western Great Plains Foothill and Piedmont Gra...,7147,CES303.817,722856,Western Great Plains Foothill and Piedmont Gra...,...,LC,NT,LC,VU,G3,218,238,243,2,0.10
mesic,224,7322,1090956,98186,982,Crowley's Ridge Mesic Loess Slope Forest,7322,CES203.079,798100,Crowley's Ridge Mesic Loess Slope Forest,...,EN,VU,CR,CR (EN-CR),G1G2,144,201,143,1,0.05
bluff,229,7327,2050154,184514,1845,East Gulf Coastal Plain Northern Loess Bluff F...,7327,CES203.481,723105,East Gulf Coastal Plain Northern Loess Bluff F...,...,LC,VU-EN,EN,EN,G2,149,143,26,1,0.05
pine,244,7346,5015841,451426,4514,Atlantic Coastal Plain Fall-line Sandhills Lon...,7346,CES203.254,723231,Atlantic Coastal Plain Fall-line Sandhills Lon...,...,LC,VU,CR,EN (EN-CR),G1G2,70,96,32,1,0.05
tallgrass,314,7421,10225903,920331,9203,Central Tallgrass Prairie,7421,CES205.683,722976,Central Tallgrass Prairie,...,LC,DD,CR,CR,G1,243,201,28,1,0.05
dune,323,7431,19717,1775,18,Southwest Florida Dune and Coastal Grassland,7431,CES203.539,723063,Southwest Florida Dune and Coastal Grassland,...,LC,NE,CR,CR,G1,245,252,179,1,0.05
dome,335,7447,900234,81021,810,South Florida Cypress Dome,7447,CES411.365,723151,South Florida Cypress Dome,...,LC,DD,LC,VU,G3,54,163,120,2,0.10
marsh,676,9197,1634510,147106,1471,Northern Atlantic Coastal Plain Tidal Salt Marsh,9197,CES203.519,723073,Northern Atlantic Coastal Plain Tidal Salt Marsh,...,LC,DD,EN,EN,G2,131,173,223,1,0.05


#### FUTURE UPDATE (CREATE ISSUE?) 

Currently our code will work with the ecosystem raster and hex files that Lana 
created in ArcGIS using the ArcMarxan plugin.  Ultimately we hope to work directly
with the full Landfire EVT 2020 raster, but the file is proving too large to 
effectively manage with our personal laptops. A solution may be found using the 
2016 Landfire data which has an available API (the 2020 data is scheduled to be 
published to the API later this year). An alternitive solution may be found using 
Dask.

If/When our code can access the full CONUS raster, the source data in repo assets 
(and links that file in this code) will need to be updated to the full version of 
the raster's LF_2020_EVT_README file. Once that occurs, we could ask for user 
input to get entries matching the 'Values' column in that file, as a way of 
selecting specific ecosystems from the full Landfire EVT 2020 data. That user 
input would be assigned to a list variable 'value_filter'. 

The user would then be prompted for a one-word 'Short_Name' value for each 
ecosystem being analyzed (ex. mesic, dune, dome), to be used in file naming. This 
abbreviated name would be added to the ecoinfo_df.  

Currently the 'Short_Name' values have been hardcoded, to match what Lana chose 
when creating her ArcGis files. The 'value_filter variable will also be hardcoded,
to match the values seen in the LF_2020_EVT_README file for the 3 ecosystems we
are using as test data (Crowley's Ridge Mesic Loess Slope Forest, Southwest 
Florida Dune and Coastal Grassland, and South Florida Cypress Dome).


In [5]:
# Create lists that will be used later in iteration loops

# Create list of threshold values to test
test_threshold = [1.0, 0.75, 0.50, 0.25]

# Define list variable 'value_filter' to show the values matching the 'Values' 
# column of ecoinfo_df for the three ecosystems which have shp and hex files 
# uploaded to the GitHub repository - 'dome', 'dune', and 'mesic', 
value_filter = [7431, 7322, 7447]

# use value_filter to create a new df with only matching records
eco_subset_df = ecoinfo_df[ecoinfo_df['Value'].isin(value_filter)]

# Create alphabetical list of ecosystems to be analyzed, taken from the 
# 'Short_Name' column of eco_subset_df
eco_list = eco_subset_df.index.values.tolist()
eco_list.sort()
# print(eco_list)
eco_list

# print(eco_subset)

['dome', 'dune', 'mesic']

#### Loop through the eco_list, create directories and input files needed by Marxan.

Each time the code runs, a new timestamped diretory is created. Inside will
be subdirectories created from the 'Short_Name' value of selected 
ecosystems. Each of these ecosystem subdirectories will have a named input 
and output folder to store the files needed by and created from the marxan 
analysis.

Functions will be written to create each specific input file needed for 
Marxan analysis. A series of five input files are needed, showing information 
about the ecosystem's hex file. Initially these formulas will use Lana's hex 
files that have been uploaded to the GitHub repository.  Eventually the formulas 
could work with files that are obtained programatically using the Landfire API (or
a future user's own source files)


In [7]:
newdir = os.path.normpath(os.path.join(data_path, 
                                       datetime.datetime.now().strftime(
                                           '%Y%m%d_%H%M%S')))
os.makedirs(newdir)
os.chdir(newdir)

for eco in eco_list:
    os.makedirs(eco),
    eco_data_path = os.path.normpath(os.path.join(data_path, newdir, eco))
    os.chdir(eco_data_path)
    os.makedirs('input')
    eco_input_data_path = os.path.normpath(os.path.join(eco_data_path, 'input'))
    os.chdir(eco_input_data_path)
    ks.get_marxan_input_files(eco)
#     inputfile_ls = ["bound.dat", "pu.dat", "puvsp.dat", "spec.dat"]
#     for file in inputfile_ls:
#         urltext = "https://raw.githubusercontent.com/csandberg303/kba-threshold-sensitivity-analysis/main/assets/data/marxan_input/"
#         url = urltext + eco + "/" + file
#         fileinfo = requests.get(url).content
#         # Reading the downloaded content and turning it into a pandas dataframe
#         fileinfo_df = pd.read_csv(io.StringIO(fileinfo.decode('utf-8')))
#         filename = file
#         output = fileinfo_df.to_csv(file)
    os.chdir(eco_data_path)
    os.makedirs('output')
    ks.create_targets_files(eco, test_threshold, ecoinfo_df, eco_data_path)
    os.chdir(newdir)
os.getcwd()



'C:\\Users\\cwsnd\\earth-analytics\\data\\kba_thresh_sa\\20220522_230233'

In [ ]:
# STEP 1:
# Create a timestamped directory, with a subdirectory for each ecosystem 
# selected in 'value_filter'.  Each ecosystem directory includes the input and 
# output directories (currently also creates the 'targets.csv' files for each 
# value in 'test_threshold' list variable, which are used by CLUZ in QGIS. 
# These files are created using the function 'create_targets_files', but may 
# not ultimately be needed by Marxan/marxanconpy)

newdir = os.path.normpath(os.path.join(data_path, 
                                       datetime.datetime.now().strftime(
                                           '%Y%m%d_%H%M%S')))
os.makedirs(newdir)
os.chdir(newdir)
for eco in eco_list:
    os.makedirs(eco),
    eco_data_path = os.path.normpath(os.path.join(data_path, newdir, eco))
    os.chdir(eco_data_path)
    os.makedirs('input')
    # IN THE FUTURE - os.chdir to 'input', and write add'l functions to create 
    # the input files needed for later analysis using marxanconpy.  Lana has
    # generated these input files with ArcGIS using the add-in ArcMarxan.  
    # we may be able to get the python code for creating these input files 
    # from QGIS using the add-in QMarxan - which is the open source version of 
    # the ArcMarxan tool Lana used.
    os.makedirs('output')
    ks.create_targets_files(eco, test_threshold, ecoinfo_df, eco_data_path)
    os.chdir(newdir)
    
os.getcwd()

#### FURTHER STEPS:
Once directories & input files are set-up as needed for Marxan analysis -
the next section of needed code are functions using actual marxanconpy, and
the handling of the results 
(store files, add collected info (and summary info?) to dataframe, which 
will ultimately be saved as .csv output file)

#### THIS CELL SHOWS ORIGINAL WORKFLOW IDEA FOR REFERENCE -

GET LIST OF ECOSYSTEMS TO ANALYZE 
    (FROM PROVIDED 'LF_EVT_2020' FILE)
    
SET UP FOLDER STRUCTURE FOR EACH ECOSYSTEM (CREATE INPUT/OUTPUT FOLDERS ETC)
    (I'VE GOT SOME INITIAL CODE FOR THIS IN A JUPYTER NOTEBOOK ON OUR 
    NATURESERVE REPOSITORY)
     
GET TWO INITIAL SHAPEFILES FOR EACH ECOSYSTEM THAT WILL BE NEEDED FOR MARXAN 
ANALYSIS, SAVE TO APPROPRIATE FOLDER 
    (USE LANA'S EXISTING FILES FROM ARCGIS WITH PAT'S 2020 DATA, OR 
    INVESTIGATE LANDFIRE API FOR 2016 DATA)
     
SET UP LOGIC OF CURRENT KBA THRESHOLDS
    (FOR IUCN A2:THREATENED ECOSYSTEM TYPE
         IF ECOSYSTEM'S 'LF_EVT_2020[RLE_FINAL]' VALUE = CR, CR (EN-CR), 
                 EN or EN (EN-CR)
             THRESHOLD = >= 5% OF ECOSYSTEM TOTAL EXTENT 
                (USE 30M, US-hectare, or US_km2 COLUMN OF 'LF_EVT_2020')
         IF ECOSYSTEM'S 'LF_EVT_2020[RLE_FINAL]' VALUE = VU
             THRESHOLD = >= 10% OF ECOSYSTEM TOTAL EXTENT

CREATE LIST FOR THRESHOLD TEST (1.0, 0.75, 0.50, 0.25)
     
*INITIAL OUTPUT* - create table of 4 threshold values for each ecosystem    
     (from Pat's Study Plan - "This will result in a tabular summary of 4 
         threshold values per ecosystem type, depending on its RLE status")
     
ASSIGN EACH ECOSYSTEM TO A SPATIAL CHARACTERISTIC TYPE (PATCHY, LINEAR or 
    MATRIX) - THIS IS BASED UPON OUR VISUAL ASSESSMENT, WHICH WILL BE 
    APPROVED BY PAT)
     
ITERATION LOOP:
    FOR EACH ECOSYSTEM IN LIST
         RUN MARXANCONPY ANALYSIS (***see note below) FOR THE APPROPRIATE 
            KBA THRESHOLD, BASED ON 'RLE_FINAL'
         COMPLETE SEPARATE RUNS FOR EACH TEST LEVEL (1.0, 0.75, 0.50, 0.25)
         SAVE RESULTS FROM EACH RUN TO DATAFRAME
    SAVE DATAFRAME TO TABLE (INCL. THE ASSIGNED SPATIAL CHARACTERISTIC INFO)

SAVE FINAL DATAFRAME AS CSV
     
     
*** USING MARXANCONPY FOR FURTHER ANALYSIS
     
     INFO FROM MARXANCONPY DOCUMENTATION - 
        https://remi-daigle.github.io/marxanconpy/
        The first step is to import the package and create a MarCon dictionary.
             import marxanconpy
             import os
             project = marxanconpy.marcon.new_project()
     
     INFO FROM https://marxanconnect.ca/tutorial.html#4_marxanconpy_Python_Module
         Using the spatial dependencies approach using landscape-based data
         https://marxanconnect.ca/targets.html
     


     
